In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.contrib import rnn

In [2]:
data = pd.read_csv("/home/ace/vertika/test/train.csv")


train_x = data.values[:,1:]
train_y = data.values[:,0]
print(train_x)
print(train_y)
split_size = int(train_x.shape[0]*0.7)
print(split_size)

train_x, val_x = train_x[:split_size, : ], train_x[split_size: , :]
print(train_x.shape)
print(len(train_x))
print(val_x.shape)

train_y, val_y = train_y[:split_size], train_y[split_size:]
print(train_y.shape)
print(val_y.shape)

test_x = pd.read_csv( "/home/ace/vertika/test/test.csv"  ).values
test_x.shape

seed = 128
rng = np.random.RandomState(seed)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 0 1 ... 7 6 9]
29399
(29399, 784)
29399
(12601, 784)
(29399,)
(12601,)


In [3]:
time_steps=28
num_units=128        #hidden LSTM units
n_input=28           #rows of 28 pixels
learning_rate=0.001  #learning rate for adam
n_classes=10         #mnist is meant to be classified in 10 classes(0-9).
batch_size=256       #size of batch

epochs = 50

In [4]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    if dataset_name == 'train':
        batch_x = train_x[batch_mask]
        batch_x = preproc(batch_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        
        batch_y = train_y[batch_mask]
        batch_y = dense_to_one_hot(batch_y)
        return batch_x, batch_y
    elif dataset_name == 'val':
        batch_x = preproc(val_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        
        batch_y = val_y
        batch_y = dense_to_one_hot(batch_y)
        return batch_x, batch_y
    else:
        batch_x = preproc(test_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        return batch_x
    

In [8]:
tf.reset_default_graph()

out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

x=tf.placeholder("float",[None,time_steps,n_input])
print(x)
y=tf.placeholder("float",[None,n_classes])
print(y)

input=tf.unstack(x ,time_steps,1)
print(input)

lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

prediction=tf.matmul(outputs[-1],out_weights)+out_bias

Tensor("Placeholder:0", shape=(?, 28, 28), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
[<tf.Tensor 'unstack:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:4' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:5' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:6' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:7' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:8' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:9' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:10' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:11' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:12' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:13' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:14' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:15' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:16' shape=(

In [6]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for ep in range(epochs):
        avg_cost = 0
        total_batch = len(train_x)/batch_size
        print("Next epoch")
        for bt in range(total_batch):
            batch_x, batch_y = batch_creator( batch_size, len(train_x), 'train')
            _, c = sess.run( [ opt, loss ] ,  feed_dict = { x:batch_x, y: batch_y } )
            avg_cost += c / total_batch
            
            if bt %100==0:
                acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
                los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
                print("Iter "+ str(bt) + "  Accuracy "+ str(acc) +  "  Loss "+ str(los) )

                #calculating test accuracy
                batch_x, batch_y = batch_creator( len(val_x) , len(val_x), 'val')
                print(batch_x.shape)
                print("Validation Accuracy:", sess.run(accuracy, feed_dict={x:batch_x, y: batch_y}))
   
    
    
    batch_x = batch_creator( len(test_x) , len(test_x), 'test')
    print(len(batch_x))
    predict = tf.argmax(prediction, 1)
    print(predict)
    pred = predict.eval({x: batch_x})
    
    print("len of pred:" + str(len(pred)))
    
    outFile = open("/home/ace/vertika/test/out.csv", "w")
    outFile.write("ImageId,Label\n")
    for i in range( len(pred)):
        outFile.write( str(i+1) + "," + str(pred[i]) + "\n")
        
    print("File written\n")

Next epoch
Iter 0  Accuracy 0.1015625  Loss 2.624053
(12601, 28, 28)
('Validation Accuracy:', 0.094198875)
Iter 100  Accuracy 0.21875  Loss 2.156833
(12601, 28, 28)
('Validation Accuracy:', 0.21958575)
Next epoch
Iter 0  Accuracy 0.24609375  Loss 2.0871425
(12601, 28, 28)
('Validation Accuracy:', 0.22942623)
Iter 100  Accuracy 0.2734375  Loss 2.0611854
(12601, 28, 28)
('Validation Accuracy:', 0.2797397)
Next epoch
Iter 0  Accuracy 0.24609375  Loss 2.0255857
(12601, 28, 28)
('Validation Accuracy:', 0.27870804)
Iter 100  Accuracy 0.28515625  Loss 1.9822577
(12601, 28, 28)
('Validation Accuracy:', 0.2870407)
Next epoch
Iter 0  Accuracy 0.30078125  Loss 1.9801004
(12601, 28, 28)
('Validation Accuracy:', 0.3070391)
Iter 100  Accuracy 0.29296875  Loss 1.8960422
(12601, 28, 28)
('Validation Accuracy:', 0.31505436)
Next epoch
Iter 0  Accuracy 0.27734375  Loss 1.9184474
(12601, 28, 28)
('Validation Accuracy:', 0.32814857)
Iter 100  Accuracy 0.2890625  Loss 1.866951
(12601, 28, 28)
('Validation 

(12601, 28, 28)
('Validation Accuracy:', 0.40941194)
Iter 100  Accuracy 0.46875  Loss 1.4887909
(12601, 28, 28)
('Validation Accuracy:', 0.41147527)
Next epoch
Iter 0  Accuracy 0.4453125  Loss 1.562899
(12601, 28, 28)
('Validation Accuracy:', 0.4044917)
Iter 100  Accuracy 0.45703125  Loss 1.5538294
(12601, 28, 28)
('Validation Accuracy:', 0.41266567)
Next epoch
Iter 0  Accuracy 0.42578125  Loss 1.5162632
(12601, 28, 28)
('Validation Accuracy:', 0.4089358)
Iter 100  Accuracy 0.453125  Loss 1.610425
(12601, 28, 28)
('Validation Accuracy:', 0.40996745)
Next epoch
Iter 0  Accuracy 0.43359375  Loss 1.5203146
(12601, 28, 28)
('Validation Accuracy:', 0.40679312)
Iter 100  Accuracy 0.453125  Loss 1.4682798
(12601, 28, 28)
('Validation Accuracy:', 0.4089358)
Next epoch
Iter 0  Accuracy 0.4765625  Loss 1.4688022
(12601, 28, 28)
('Validation Accuracy:', 0.41314182)
Iter 100  Accuracy 0.3984375  Loss 1.5760247
(12601, 28, 28)
('Validation Accuracy:', 0.41195142)
Next epoch
Iter 0  Accuracy 0.45312